<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [1]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
# !pip install hazm
# !pip install unidecode
# !pip install pandas
# !pip install nltk

In [2]:
import os
import re
import csv
import json
import hazm
import nltk
import pickle
import zipfile
import requests
import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [3]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [3]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [4]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [6]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [7]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [8]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]


def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم.
    </p>
</div>

In [9]:
def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


preprocessor = Preprocessor(stopwords_path='stopwords.txt')

# texts = [data_to_text(data) for _, data in dataset.iterrows()]
# preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
# save_preprocessed_texts(preprocessed_texts)

preprocessed_texts = load_preprocessed_texts()

# Models
## Boolean

In [19]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts

mini_dataset, mini_preprocessed_texts = get_mini_dataset()

In [22]:
class BooleanIR:

    def __init__(self):
        self.vectorizer = CountVectorizer(binary=True)
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def preprocess_query(self, query):
        preprocessor = Preprocessor(stopwords_path='stopwords.txt')
        preprocessed_query = []
        for subquery in query:
            if subquery[0] == '-':
                preprocessed_query.append('-' + (' '.join(preprocessor.preprocess(subquery[1:]))))
            else:
                preprocessed_query.append(' '.join(preprocessor.preprocess(subquery)))
        return preprocessed_query

    def predict(self, query, dataset, k):
        query = self.preprocess_query(query)
        query_result = []
        query_result_BUT = []
        for subquery in query:
            is_subquery_but = False
            if subquery[0] == '-':
                is_subquery_but = True
                subquery = subquery[1:]
            query_transform = self.vectorizer.transform([subquery]).todense().tolist()[0]
            query_indices = np.nonzero(query_transform)[0]
            for index, doc in self.dense_vectors_df.iterrows():
                if all(np.take(doc, query_indices)):
                    if is_subquery_but:
                        query_result_BUT.append(index)
                    else:
                        query_result.append(index)
        query_result = list(set(query_result) - set(query_result_BUT))
        k = min(k, len(query_result))
        return dataset.iloc[query_result[:k]]


def save_boolean_model(model, path="BooleanIR_model.pickle"):
    with open(path, "wb") as file:
        pickle.dump(model, file)


def load_boolean_model(path="BooleanIR_model.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)

In [15]:
# booleanIR_model = BooleanIR()
# booleanIR_model.fit_transform_vectorizer(mini_preprocessed_texts)
# save_boolean_model(booleanIR_model)

booleanIR_model = load_boolean_model()

100%|██████████| 4000/4000 [00:20<00:00, 198.12it/s]


In [25]:
query = ["آیفون", "-اپل", "-شرکت", "-کمپانی"]
booleanIR_model.predict(query, mini_dataset, k=10)

,date,title,intro,body,category
3715,دوشنبه ۲۴ آبان ۱۴۰۰ - ۰۰:۰۳,واتس‌اپ خود را با یک راهکار ساده ایمن‌تر کنید,واتس‌اپ برنامه پیام رسانی است که خود را بسیار ...,به گزارش همشهری آنلاین به نقل از ایسنا، با این...,LifeSkills
3505,جمعه ۱۵ بهمن ۱۴۰۰ - ۲۰:۳۱,تصاویر | آیفون ۷۶۰ میلیون تومانی؛ ویژه بازی‌ها...,بازی‌های المپیک زمستانی ۲۰۲۲ پکن در حال برگزار...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
3571,شنبه ۲۵ دی ۱۴۰۰ - ۱۱:۲۲,دوربین این گوشی از آیفون ۱۳ هم بهتر است,موسسه DxOMark به تازگی بررسی کامل دوربین موبای...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT
1558,چهارشنبه ۱۱ خرداد ۱۴۰۱ - ۱۰:۴۹,دزد موبایل برای فروش گوشیِ سرقتی آگهی داد!,سارقان موبایل که برای فروش گوشیِ سرقت‌شده در ف...,به گزارش همشهری آنلاین به نقل از روزنامه ایران...,Society
3513,شنبه ۹ بهمن ۱۴۰۰ - ۰۰:۱۸,شباهت عجیب یک گوشی چینی به آیفون ۱۳,یک دستگاه بسیار عجیب و غریب دیروز در چین با نا...,به گزارش همشهری آنلاین و به نقل از وب‌سایت جی‌...,IT


## TF-IDF

In [40]:
class TF_IDF:

    def __init__(self):
        self.vectorizer = TfidfVectorizer()
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def predict(self, query, dataset, k):
        query = ' '.join(Preprocessor(stopwords_path='stopwords.txt').preprocess(query))
        query_transform = self.vectorizer.transform([query]).todense().tolist()[0]
        dense_vectors = self.dense_vectors_df.values.tolist()
        df_cosine_sim = list(map(lambda doc: self.cosine_sim(query_transform, doc), dense_vectors))
        self.dense_vectors_df['query_sim'] = df_cosine_sim
        indices = self.dense_vectors_df.nlargest(k, 'query_sim').index
        self.dense_vectors_df = self.dense_vectors_df.drop(columns=['query_sim'])
        return dataset.iloc[indices]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))


def save_TF_IDF_model(model, path="TF_IDF_model.pickle"):
    with open(path, "wb") as file:
        pickle.dump(model, file)


def load_TF_IDF_model(path="TF_IDF_model.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


In [41]:
TF_IDF_model = TF_IDF()
TF_IDF_model.fit_transform_vectorizer(mini_preprocessed_texts)
save_TF_IDF_model(TF_IDF_model)

# TF_IDF_model = load_TF_IDF_model()

100%|██████████| 4000/4000 [00:21<00:00, 189.17it/s]


In [42]:
query = "روسیه به اوکراین حمله کرد"
TF_IDF_model.predict(query, mini_dataset, k=10)

,date,title,intro,body,category
733,پنجشنبه ۵ اسفند ۱۴۰۰ - ۱۰:۲۸,ببینید | اوکراین؛ از آژیر حمله هوایی تا آتش سو...,رئیس جمهوری روسیه صبح امروز اعلام کرد که در پا...,به گزارش همشهری آنلاین به نقل از تسنیم، شورای ...,World
501,یکشنبه ۱ اسفند ۱۴۰۰ - ۰۸:۴۹,هشدار درباره نزدیکی جنگ جهانی سوم | روسیه قصد ...,بوریس جانسون، نخست وزیر انگلیس هشدار داد که وض...,به گزارش همشهری آنلاین به نقل از ایسنا، جانسون...,World
444,دوشنبه ۲۵ بهمن ۱۴۰۰ - ۰۸:۲۴,روسیه ژست حمله به اوکراین را گرفت | شاید چین ه...,یک استاد ژئوپلیتیک دانشگاه با اشاره به اینکه ر...,به گزلرش همشهری آنلاین، عبدالرضا فرجی راد دربا...,World
3271,سه‌شنبه ۲۰ اردیبهشت ۱۴۰۱ - ۱۷:۱۳,حمله سایبری به اینترنت ماهواه‌ای کار روسیه بود...,اتحادیه اروپا مدعی شد که حمله سایبری بزرگ علیه...,به گزارش همشهری آنلاین و به نقل خبرگزاری رویتر...,IT
571,یکشنبه ۲۴ بهمن ۱۴۰۰ - ۱۱:۲۹,حمله قریب‌الوقوع روسیه به اوکراین؟,یک مقام کاخ الیزه می‌گوید در تماس تلفنی رئیس ج...,به گزارش همشهری آنلاین به نقل از فارس، یک مقام...,World
760,سه‌شنبه ۱۰ اسفند ۱۴۰۰ - ۱۳:۱۱,تازه‌ترین درخواست وزیر امور خارجه اوکراین علیه...,دیمیترو کولبا، وزیر امور خارجه اوکراین، پس از ...,به گزارش همشهری آنلاین و به نقل از گاردین، او ...,World
764,جمعه ۶ اسفند ۱۴۰۰ - ۱۲:۱۹,ادعای وزیر دفاع انگلیس درباره نقشه روسیه برای ...,وزیر دفاع انگلیس در بیانیه‌ای مدعی شد، روسیه ق...,به گزارش همشهری آنلاین و به نقل از گاردین، بن ...,World
578,جمعه ۶ اسفند ۱۴۰۰ - ۱۶:۰۷,پوتین: آماده مذاکره با اوکراین هستیم | موضع ری...,رئیس جمهور چین در یک تماس تلفنی با رئیس جمهور ...,به گزارش همشهری‌آنلاین به نقل از فارس، رسانه‌ه...,World
589,شنبه ۱۷ اردیبهشت ۱۴۰۱ - ۱۶:۲۷,جنگ اوکراین روزانه چقدر برای روسیه آب می‌خورد؟,یک نشریه آمریکایی در گزارشی با اشاره به سومین ...,به گزارش همشهری‌آنلاین، ایرنا به نقل از نشریه ...,World
460,یکشنبه ۸ اسفند ۱۴۰۰ - ۱۶:۱۰,منتظر جنگ جهانی سوم باشیم؟ | امکان چریکی شدن ج...,جهانگیر کرمی، عضو هیات علمی دانشگاه تهران و کا...,همشهری‌آنلاین - اصغر صوفی: درگیری میان اوکراین...,World


## Fasttext

In [22]:
mini_dataset, mini_preprocessed_texts = get_mini_dataset()

100%|██████████| 4000/4000 [00:19<00:00, 202.72it/s]


In [190]:
class FastText:

    def __init__(self, method='skipgram'):
        self.method = method
        self.mean_embed = []
        self.model = None

    def train(self, preprocessed_text):
        with open('fasttext_train.txt', 'w', encoding='utf-8') as file:
            file.write('\n'.join(list(map(lambda doc: ' '.join(doc), preprocessed_text))))
        self.model = fasttext.train_unsupervised('fasttext_train.txt', self.method, minn=2, maxn=5, wordNgrams=10)
        os.remove('fasttext_train.txt')
        for doc in preprocessed_text:
            embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), doc)), axis=0)
            self.mean_embed.append(embed)
        self.mean_embed = np.array(self.mean_embed)

    def predict(self, query, dataset, k):
        query_embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), query)), axis=0)
        dataset_sim = np.array(list(map(lambda doc: self.cosine_sim(query_embed, doc), self.mean_embed)))
        idx = np.argsort(-dataset_sim)
        return dataset.iloc[list(idx[:k])]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_FastText_model(self, path='FastText_model.bin'):
        self.model.save_model(path)
        np.save('FastText_mean_embed.npy', self.mean_embed)

    def load_FastText_model(self, path="FastText_model.bin"):
        self.model = fasttext.load_model(path)
        self.mean_embed = np.load('FastText_mean_embed.npy')

In [191]:
FastText_model = FastText()
FastText_model.train(mini_preprocessed_texts)

Read 0M words
Number of words:  16628
Number of labels: 0
Progress: 100.0% words/sec/thread:   34163 lr:  0.000000 avg.loss:  1.983081 ETA:   0h 0m 0s 17.8% words/sec/thread:   30396 lr:  0.041121 avg.loss:  2.059621 ETA:   0h 0m11s 45.2% words/sec/thread:   33060 lr:  0.027419 avg.loss:  2.016508 ETA:   0h 0m 6s  0h 0m 0s


In [192]:
#FastText_model.save_FastText_model()
#FastText_model.load_FastText_model()

In [193]:
semantically_similar_words = FastText_model.model.get_nearest_neighbors('آیفون')
print(semantically_similar_words)

[(0.8355935215950012, 'آی\u200cپد'), (0.8261321783065796, 'اپل'), (0.7838040590286255, 'گالکسی'), (0.7690565586090088, 'آیفون\u200cهای'), (0.7669818997383118, 'wwdc'), (0.7611461281776428, 'اسنپ\u200cدراگون'), (0.7572290897369385, 'سامسونگ'), (0.7492721080780029, 'tsmc'), (0.748271644115448, 'هدفون'), (0.7474921941757202, 'bionic')]


In [194]:
queries = []
queries.append("روسیه به اوکراین حمله کرد")
queries.append(" آیفون ")
queries.append(' اسکوچیچ نگران شد ')
queries.append(' گلکسی سامسونگ ')
for query in queries:
    preprocessed_query = ' '.join(preprocessor.preprocess(query))
    display(FastText_model.predict(preprocessed_query, mini_dataset, k=10))


,date,title,intro,body,category
2905,دوشنبه ۱ فروردین ۱۴۰۱ - ۱۲:۰۰,زندگی عجیب ۲ نابغه شیدای موسیقی؛ دَلی بویوک آق...,بویوک‌آقا مراغه‌ای و رضا دیوونه تهرانی در میان...,به گزارش همشهری‌آنلاین به نقل از روزنامه هفت ص...,Culture
2471,سه‌شنبه ۱ تیر ۱۴۰۰ - ۱۰:۵۶,بندهای رنگی,کتاب پیشنهادی برای مطالعه در پایان این هفته، ر...,همشهری آنلاین- علی‌الله سلیمی: کتاب پیشنهادی ب...,Science
3825,دوشنبه ۲ فروردین ۱۴۰۰ - ۰۷:۲۳,پیشنهادهای اکبر اکسیر برای دوم فروردین | در رو...,اکبر اکسیر، شاعر و طنزپرداز که صاحب مجموعه‌های...,به گزارش همشهری آنلاین او در دوم فروردین، روز ...,LifeSkills
2947,پنجشنبه ۱۸ فروردین ۱۴۰۱ - ۱۰:۳۰,طعنه تلخ و سنگین بازیگر زن سینما به بهاره رهنم...,پرستو گلستانی در اینستاگرامش، از این‌که بهاره ...,به گزارش همشهری‌آنلاین، پرستو گلستانی، بازیگر ...,Culture
3023,یکشنبه ۲۱ فروردین ۱۴۰۱ - ۱۶:۵۳,ماجرای معتادی که از شهید آوینی پول می‌گرفت,سایت خاطره نگاری به مناسبت روز قلم خاطره ای از...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، خا...,Culture
2867,جمعه ۱۴ آبان ۱۴۰۰ - ۱۴:۵۳,راپونزل گمنام است یا رودابه؟,«فرزندان ایران راپونزل را بیشتر می‌شناسند یا ر...,به گزارش همشهری آنلاین، صفحه این گروه دراین‌با...,Culture
3854,یکشنبه ۸ خرداد ۱۴۰۱ - ۰۹:۳۲,تصویر جایی که ناصرالدین ‌شاه بهشت نامیدش,ناصرالدین قاجار پیش از آنکه سفرهای فرنگ برود د...,به گزارش همشهری آنلاین، روایت ناصرالدین قاجار ...,LifeSkills
3956,چهارشنبه ۲۰ بهمن ۱۴۰۰ - ۱۸:۵۶,ببینید | ظروف غذاخوری فوق لاکچری با طراحی متفا...,زمستان امسال هرمس از ظروف غذاخوری چینی جدید خو...,به گزارش همشهری آنلاین به نقل از ایندیپندنت، ز...,LifeSkills
1470,سه‌شنبه ۹ شهریور ۱۴۰۰ - ۱۱:۳۰,سنگ‌های لق روی دیوار,کتاب پیشنهادی برای مطالعه در پایان این هفته، ا...,همشهری آنلاین- علی‌الله سلیمی: کتاب پیشنهادی ب...,Society
3621,پنجشنبه ۸ اردیبهشت ۱۴۰۱ - ۱۷:۲۴,تصویر حیرت‌انگیز از یک کباب فروش برنده مسابقه ...,عکسی از یک فروشنده غذای خیابانی که در حال پخت ...,به گزارش همشهری آنلاین، عکاس هندی این عکس را د...,LifeSkills


,date,title,intro,body,category
3023,یکشنبه ۲۱ فروردین ۱۴۰۱ - ۱۶:۵۳,ماجرای معتادی که از شهید آوینی پول می‌گرفت,سایت خاطره نگاری به مناسبت روز قلم خاطره ای از...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، خا...,Culture
3621,پنجشنبه ۸ اردیبهشت ۱۴۰۱ - ۱۷:۲۴,تصویر حیرت‌انگیز از یک کباب فروش برنده مسابقه ...,عکسی از یک فروشنده غذای خیابانی که در حال پخت ...,به گزارش همشهری آنلاین، عکاس هندی این عکس را د...,LifeSkills
3854,یکشنبه ۸ خرداد ۱۴۰۱ - ۰۹:۳۲,تصویر جایی که ناصرالدین ‌شاه بهشت نامیدش,ناصرالدین قاجار پیش از آنکه سفرهای فرنگ برود د...,به گزارش همشهری آنلاین، روایت ناصرالدین قاجار ...,LifeSkills
3055,سه‌شنبه ۲۳ فروردین ۱۴۰۱ - ۱۷:۰۳,دانلود آهنگ جدید آرون افشار عطر بهار تیتراژ بر...,"آهنگ جدید ""عطر بهار"" با صدای ""آرون افشار"" منتش...","""وحید افشار"" یا ""آرون افشار"" متولد ۱ تیر ۱۳۷۷ ...",Culture
3152,دوشنبه ۱۹ اردیبهشت ۱۴۰۱ - ۱۲:۳۱,واکنش کیومرث پوراحمد به ماجرای نمایش فیلم احمد...,کیومرث پوراحمد پیرامون نمایش فیلم داستان­های ه...,به گزارش همشهری آنلاین به نقل از ایسنا، کیومرث...,Culture
2471,سه‌شنبه ۱ تیر ۱۴۰۰ - ۱۰:۵۶,بندهای رنگی,کتاب پیشنهادی برای مطالعه در پایان این هفته، ر...,همشهری آنلاین- علی‌الله سلیمی: کتاب پیشنهادی ب...,Science
2883,پنجشنبه ۴ فروردین ۱۴۰۱ - ۱۳:۴۱,هفت‌سین از کجا به خانه ایرانی‌ها وارد شد؟,«مطالعه تاریخچه ⁠⁣آئین هفت‌سین چینی نشان می‌ده...,به گزارش همشهری‌آنلاین به نقل از ایسنا، در نو...,Culture
2836,شنبه ۲۵ اردیبهشت ۱۴۰۰ - ۱۴:۰۷,انتشار کتاب نخستین جشنواره ملی عکس شاهنامه,کتاب نخستین جشنواره ملی عکس شاهنامه در این مجم...,به گزارش همشهری آنلاین به نقل از ایبنا، نسخه ک...,Culture
2905,دوشنبه ۱ فروردین ۱۴۰۱ - ۱۲:۰۰,زندگی عجیب ۲ نابغه شیدای موسیقی؛ دَلی بویوک آق...,بویوک‌آقا مراغه‌ای و رضا دیوونه تهرانی در میان...,به گزارش همشهری‌آنلاین به نقل از روزنامه هفت ص...,Culture
2113,جمعه ۲۹ مرداد ۱۴۰۰ - ۱۹:۴۳,پرز دست بردار امباپه نیست؛ مذاکره مستقیم با ام...,برای جذب کیلیان امباپه، رئال مادرید باید آل ثا...,به گزارش همشهری آنلاین به نقل از ورزش۳ تمیم بن...,Sport


,date,title,intro,body,category
3956,چهارشنبه ۲۰ بهمن ۱۴۰۰ - ۱۸:۵۶,ببینید | ظروف غذاخوری فوق لاکچری با طراحی متفا...,زمستان امسال هرمس از ظروف غذاخوری چینی جدید خو...,به گزارش همشهری آنلاین به نقل از ایندیپندنت، ز...,LifeSkills
2471,سه‌شنبه ۱ تیر ۱۴۰۰ - ۱۰:۵۶,بندهای رنگی,کتاب پیشنهادی برای مطالعه در پایان این هفته، ر...,همشهری آنلاین- علی‌الله سلیمی: کتاب پیشنهادی ب...,Science
2905,دوشنبه ۱ فروردین ۱۴۰۱ - ۱۲:۰۰,زندگی عجیب ۲ نابغه شیدای موسیقی؛ دَلی بویوک آق...,بویوک‌آقا مراغه‌ای و رضا دیوونه تهرانی در میان...,به گزارش همشهری‌آنلاین به نقل از روزنامه هفت ص...,Culture
3621,پنجشنبه ۸ اردیبهشت ۱۴۰۱ - ۱۷:۲۴,تصویر حیرت‌انگیز از یک کباب فروش برنده مسابقه ...,عکسی از یک فروشنده غذای خیابانی که در حال پخت ...,به گزارش همشهری آنلاین، عکاس هندی این عکس را د...,LifeSkills
3622,شنبه ۸ خرداد ۱۴۰۰ - ۱۱:۳۹,طرز تهیه بستنی توت‌فرنگی | ترفندی برای یخ نزدن...,بستنی توت‌فرنگی یکی از خوشمزه‌ترین و محبوب‌تری...,به گزارش همشهری آنلاین، توت‌فرنگی یکی از خوشمز...,LifeSkills
3935,شنبه ۳۰ بهمن ۱۴۰۰ - ۱۸:۱۱,طرز تهیه کیک ساده با پف زیاد | یک نکته طلایی ب...,اگر به کیک‌پزی و پخت انواع کیک علاقه‌مندید برا...,به گزارش همشهری آنلاین، کیک اسفنجی یکی از انوا...,LifeSkills
3983,جمعه ۲۹ مرداد ۱۴۰۰ - ۱۰:۵۴,برای لاغری شکم کدام ورزش مناسب است؟,برای کاهش سایز شکم و پهلو و همچنین آب کردن شکم...,به گزارش همشهری آنلاین به نقل از بهداشت نیوز، ...,LifeSkills
3992,شنبه ۹ بهمن ۱۴۰۰ - ۰۸:۰۰,۶ روش تمیزکاری رایج و اشتباه که نباید به کار ب...,گذشتگان ما برای انجام کارهای خانه و تمیزی روش‌...,به گزارش همشهری آنلاین، بهتر است نگاه دوباره‌ا...,LifeSkills
3968,دوشنبه ۱۴ تیر ۱۴۰۰ - ۱۵:۱۵,طرز تهیه رب گوجه‌فرنگی در خانه,رب گوجه‌فرنگی خانگی کاملا متفاوت و خوشمزه‌تر ا...,به گزارش همشهری آنلاین، رب گوجه‌فرنگی خانگی کا...,LifeSkills
1470,سه‌شنبه ۹ شهریور ۱۴۰۰ - ۱۱:۳۰,سنگ‌های لق روی دیوار,کتاب پیشنهادی برای مطالعه در پایان این هفته، ا...,همشهری آنلاین- علی‌الله سلیمی: کتاب پیشنهادی ب...,Society


,date,title,intro,body,category
3685,شنبه ۱۳ شهریور ۱۴۰۰ - ۰۷:۵۱,طرز تهیه ماکارونی و پنیر | در سریع‌ترین زمان م...,ماکارونی و پنیر، پاستا با سس پنیر یا مک اند چی...,به گزارش همشهری آنلاین به نقل از ایرنا، مک اند...,LifeSkills
2905,دوشنبه ۱ فروردین ۱۴۰۱ - ۱۲:۰۰,زندگی عجیب ۲ نابغه شیدای موسیقی؛ دَلی بویوک آق...,بویوک‌آقا مراغه‌ای و رضا دیوونه تهرانی در میان...,به گزارش همشهری‌آنلاین به نقل از روزنامه هفت ص...,Culture
3024,جمعه ۳ دی ۱۴۰۰ - ۱۱:۱۰,تصاویر | اقامت مسافران در غار دزد کریسمس | گذر...,گرینچ، شخصیت داستانی مشهوری که از کریسمس متنفر...,به گزارش همشهری آنلاین، دکتر سوس اینترپرایز، ب...,Culture
3935,شنبه ۳۰ بهمن ۱۴۰۰ - ۱۸:۱۱,طرز تهیه کیک ساده با پف زیاد | یک نکته طلایی ب...,اگر به کیک‌پزی و پخت انواع کیک علاقه‌مندید برا...,به گزارش همشهری آنلاین، کیک اسفنجی یکی از انوا...,LifeSkills
3956,چهارشنبه ۲۰ بهمن ۱۴۰۰ - ۱۸:۵۶,ببینید | ظروف غذاخوری فوق لاکچری با طراحی متفا...,زمستان امسال هرمس از ظروف غذاخوری چینی جدید خو...,به گزارش همشهری آنلاین به نقل از ایندیپندنت، ز...,LifeSkills
3992,شنبه ۹ بهمن ۱۴۰۰ - ۰۸:۰۰,۶ روش تمیزکاری رایج و اشتباه که نباید به کار ب...,گذشتگان ما برای انجام کارهای خانه و تمیزی روش‌...,به گزارش همشهری آنلاین، بهتر است نگاه دوباره‌ا...,LifeSkills
3622,شنبه ۸ خرداد ۱۴۰۰ - ۱۱:۳۹,طرز تهیه بستنی توت‌فرنگی | ترفندی برای یخ نزدن...,بستنی توت‌فرنگی یکی از خوشمزه‌ترین و محبوب‌تری...,به گزارش همشهری آنلاین، توت‌فرنگی یکی از خوشمز...,LifeSkills
3825,دوشنبه ۲ فروردین ۱۴۰۰ - ۰۷:۲۳,پیشنهادهای اکبر اکسیر برای دوم فروردین | در رو...,اکبر اکسیر، شاعر و طنزپرداز که صاحب مجموعه‌های...,به گزارش همشهری آنلاین او در دوم فروردین، روز ...,LifeSkills
1470,سه‌شنبه ۹ شهریور ۱۴۰۰ - ۱۱:۳۰,سنگ‌های لق روی دیوار,کتاب پیشنهادی برای مطالعه در پایان این هفته، ا...,همشهری آنلاین- علی‌الله سلیمی: کتاب پیشنهادی ب...,Society
3985,دوشنبه ۲۹ آذر ۱۴۰۰ - ۱۱:۰۲,دستور پخت یک پیتزای ساده در خانه,پیتزا یکی از غذاهای ساده و محبوب است که مدل‌ها...,به گزارش همشهری آنلاین، دستور تهیه این پیتزا ب...,LifeSkills
